In [1]:
import pandas as pd
in_out_df = pd.read_csv('aac_intakes_outcomes.csv')

### Is there an area where more pets are found?
#### Find the top 5 places where animals are found so the shelter can coordinate with local volunteers and animal control to monitor these areas.

In [5]:
location_counts = in_out_df['found_location']
top_five_locations = location_counts.value_counts()[:5]

print("Top 5 Locations where pets are found:")
top_five_locations

Top 5 Locations where pets are found:


found_location
Austin (TX)                          14311
Outside Jurisdiction                   945
Travis (TX)                            907
7201 Levander Loop in Austin (TX)      514
Del Valle (TX)                         407
Name: count, dtype: int64

### What is the average number of pets found in a month in the year 2015? Are there months where there is a higher number of animals found?
#### Knowing the number of pets the shelter might see in a month can help them gather enough resources and donations to care for the animals they receive.


### What is the ratio of incoming pets vs. adopted pets?
#### This key metric helps the shelter know how they are doing.

### What is the distribution of the types of animals in the shelter?
#### Find the count of each type of animal in the shelter.

### What are the adoption rates for specific breeds?
#### Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.

### What are the adoption rates for different colorings?
#### Find the top 5 colorings in the shelter (based on count) and then find the adoption percentage of each color.

### About how many animals are spayed/neutered each month?
#### This will help the shelter allocate resources and staff. Assume that all intact males and females will be spayed/neutered.